In [30]:
from typing import List, Dict, Set, Tuple
from itertools import product, combinations
from copy import deepcopy
import collections

class AdvancedSATSolver:
    def __init__(self, clauses: List[List[Tuple[str, bool]]]):
        """
        Initialize the SAT solver with a list of clauses.
        
        Each clause is a list of literals, where each literal is a tuple (variable, is_positive).
        Example: [
            [('x1', True), ('x2', False)],  # x1 OR (NOT x2)
            [('x2', True), ('x3', True)],   # x2 OR x3
        ]
        """
        self.original_clauses = clauses
        self.impossibility_signatures = {}  # Track impossible constraint combinations
    
    def count_variables(self, clauses: List[List[Tuple[str, bool]]]) -> Dict[str, int]:
        """
        Count the number of terms in each clause to order them.
        
        Returns a dictionary of variables and their occurrence count.
        """
        var_count = {}
        for clause in clauses:
            for var, _ in clause:
                var_count[var] = var_count.get(var, 0) + 1
        return var_count
    
    def group_clauses(self, clauses: List[List[Tuple[str, bool]]]) -> List[List[List[Tuple[str, bool]]]]:
        """
        Group clauses based on their complexity (number of terms).
        
        Returns a list of groups, where each group is sorted from least to most complex.
        """
        var_count = self.count_variables(clauses)
        
        # Sort clauses by their complexity (number of unique variables)
        sorted_clauses = sorted(
            clauses, 
            key=lambda clause: len(set(var for var, _ in clause))
        )
        
        # Group clauses with similar complexity
        grouped_clauses = []
        current_group = []
        for clause in sorted_clauses:
            if not current_group or len(current_group[-1]) == len(clause):
                current_group.append(clause)
            else:
                grouped_clauses.append(current_group)
                current_group = [clause]
        
        if current_group:
            grouped_clauses.append(current_group)
        
        return grouped_clauses
    
    def is_clause_satisfiable(self, clause: List[Tuple[str, bool]], assignment: Dict[str, bool]) -> bool:
        """
        Check if a single clause is satisfiable given a variable assignment.
        """
        for var, is_positive in clause:
            if var in assignment:
                if (assignment[var] and is_positive) or (not assignment[var] and not is_positive):
                    return True
        return len(clause) == 0
    
    def is_solution_valid(self, clauses: List[List[Tuple[str, bool]]], assignment: Dict[str, bool]) -> bool:
        """
        Check if a solution satisfies all given clauses.
        """
        return all(self.is_clause_satisfiable(clause, assignment) for clause in clauses)
    
    def generate_variable_combinations(self, variables: Set[str]) -> List[Dict[str, bool]]:
        """
        Generate all possible boolean assignments for given variables.
        
        Returns a list of possible assignments.
        """
        var_list = list(variables)
        return [
            dict(zip(var_list, combination)) 
            for combination in product([True, False], repeat=len(var_list))
        ]
    
    def explore_group_solutions(self, group_clauses: List[List[Tuple[str, bool]]]) -> List[Dict[str, bool]]:
        """
        Explore and find all possible solutions for a group of clauses.
        
        Implements the multi-solution exploration strategy.
        """
        # Extract unique variables in this group
        variables = set(var for clause in group_clauses for var, _ in clause)
        
        # Generate all possible assignments
        potential_solutions = []
        
        for assignment in self.generate_variable_combinations(variables):
            # Check if this assignment satisfies all clauses in the group
            if self.is_solution_valid(group_clauses, assignment):
                potential_solutions.append(assignment)
        
        return potential_solutions
    
    def prune_solutions(self, 
                        solutions: List[Dict[str, bool]], 
                        additional_constraints: Dict[Tuple, bool] = None) -> List[Dict[str, bool]]:
        """
        Prune solutions based on known impossible constraints.
        
        Uses the accumulated impossibility signatures to filter solutions.
        """
        if additional_constraints is None:
            additional_constraints = {}
        
        pruned_solutions = []
        for solution in solutions:
            is_valid = True
            
            # Check against impossibility signatures
            for constraint, impossibility in self.impossibility_signatures.items():
                if all(solution.get(var) == val for var, val in constraint):
                    is_valid = False
                    break
            
            # Check additional constraints
            for constraint, value in additional_constraints.items():
                if all(solution.get(var) == val for var, val in constraint):
                    is_valid = False
                    break
            
            if is_valid:
                pruned_solutions.append(solution)
        
        return pruned_solutions
    
    def solve(self) -> List[Dict[str, bool]]:
        """
        Main solving method that implements the multi-stage SAT solving strategy.
        
        Explores solutions across different complexity groups while learning and pruning.
        """
        # Group clauses by complexity
        grouped_clauses = self.group_clauses(self.original_clauses)
        
        # Start with solutions for the least complex group
        current_solutions = self.explore_group_solutions(grouped_clauses[0])
        
        # Iterate through subsequent groups
        for group in grouped_clauses[1:]:
            # Track the best solutions
            next_solutions = []
            
            # Explore solutions for each current solution
            for solution in current_solutions:
                # Explore solutions for this group
                group_solutions = self.explore_group_solutions(group)
                
                # Try to combine current solution with group solutions
                for group_solution in group_solutions:
                    # Merge solutions
                    merged_solution = {**solution, **group_solution}
                    
                    # Validate merged solution
                    if self.is_solution_valid(self.original_clauses, merged_solution):
                        next_solutions.append(merged_solution)
            
            # Update current solutions
            current_solutions = next_solutions
            
            # If no solutions remain, the problem is unsatisfiable
            if not current_solutions:
                return []
        
        return current_solutions

# Example usage and demonstration
def main():
    # Example SAT problem
    example_clauses = [
        [('x2', True)],
        [('x5', True)],
        [('x1', True), ('x2', True),('x3', True), ('x4', True),('x5', True)], 
        [('x1', False), ('x2', False),('x3', False), ('x4', True),('x5', False)],   # x1 OR (NOT x2)
        [('x1', True), ('x2', False),('x3', True), ('x5', True)],    # x2 OR x3
        [('x2', False), ('x3', True)],
          # (NOT x1) OR (NOT x3)
    ]
    
    solver = AdvancedSATSolver(example_clauses)
    solutions = solver.solve()
    
    print("Satisfiable Solutions:")
    for solution in solutions:
        print(collections.OrderedDict(sorted(solution.items())))

if __name__ == "__main__":
    main()

Satisfiable Solutions:


In [27]:
from typing import List, Dict, Set, Tuple
from itertools import product
from collections import defaultdict, OrderedDict

class SATSolver:
    def __init__(self, clauses: List[List[Tuple[str, bool]]]):
        """
        Initialize the SAT solver with a list of clauses.
        
        Each clause is a list of literals, where each literal is a tuple (variable, is_positive).
        Example: [
            [('x1', True), ('x2', False)],  # x1 OR (NOT x2)
            [('x2', True), ('x3', True)],   # x2 OR x3
        ]
        """
        self.clauses = clauses
        self.impossibility_signatures = set()  # Store invalid variable-value combinations

    def count_variable_occurrences(self) -> Dict[str, int]:
        """
        Count occurrences of each variable in the clauses for heuristic optimization.
        """
        variable_count = defaultdict(int)
        for clause in self.clauses:
            for var, _ in clause:
                variable_count[var] += 1
        return variable_count

    def group_clauses_by_size(self) -> List[List[List[Tuple[str, bool]]]]:
        """
        Group clauses by their size (number of literals in the clause).
        """
        grouped_clauses = defaultdict(list)
        for clause in self.clauses:
            grouped_clauses[len(clause)].append(clause)
        return [grouped_clauses[size] for size in sorted(grouped_clauses)]

    def is_clause_satisfied(self, clause: List[Tuple[str, bool]], assignment: Dict[str, bool]) -> bool:
        """
        Check if a single clause is satisfied given a variable assignment.
        """
        return any(
            (assignment.get(var) is not None and assignment[var] == is_positive)
            for var, is_positive in clause
        )

    def is_assignment_valid(self, assignment: Dict[str, bool]) -> bool:
        """
        Check if a variable assignment satisfies all clauses.
        """
        return all(
            self.is_clause_satisfied(clause, assignment) for clause in self.clauses
        )

    def generate_assignments(self, variables: Set[str]) -> List[Dict[str, bool]]:
        """
        Generate all possible boolean assignments for a given set of variables.
        """
        return [
            dict(zip(variables, values))
            for values in product([True, False], repeat=len(variables))
        ]

    def prune_assignments(
        self, assignments: List[Dict[str, bool]]
    ) -> List[Dict[str, bool]]:
        """
        Prune invalid assignments using impossibility signatures.
        """
        valid_assignments = []
        for assignment in assignments:
            is_valid = all(
                not all(assignment.get(var) == value for var, value in constraint)
                for constraint in self.impossibility_signatures
            )
            if is_valid:
                valid_assignments.append(assignment)
        return valid_assignments

    def solve_group(self, group: List[List[Tuple[str, bool]]]) -> List[Dict[str, bool]]:
        """
        Solve for all satisfying assignments for a specific group of clauses.
        """
        variables = {var for clause in group for var, _ in clause}
        all_assignments = self.generate_assignments(variables)
        return [assignment for assignment in all_assignments if self.is_assignment_valid(assignment)]

    def merge_assignments(
        self, current_assignments: List[Dict[str, bool]], new_assignments: List[Dict[str, bool]]
    ) -> List[Dict[str, bool]]:
        """
        Combine current assignments with new ones, ensuring consistency.
        """
        merged_assignments = []
        for current in current_assignments:
            for new in new_assignments:
                merged = {**current, **new}
                if self.is_assignment_valid(merged):
                    merged_assignments.append(merged)
        return merged_assignments

    def enforce_fixed_assignments(
        clauses: List[List[Tuple[str, bool]]]
    ) -> Tuple[List[List[Tuple[str, bool]]], Dict[str, bool]]:
        """
        Extract and enforce single literal clauses as fixed assignments.
        """
        fixed_assignments = {}
        remaining_clauses = []

        for clause in clauses:
            if len(clause) == 1:  # Single-literal clause
                var, value = clause[0]
                if var in fixed_assignments and fixed_assignments[var] != value:
                    # Contradictory fixed assignments
                    raise ValueError(f"Contradictory assignment for variable {var}.")
                fixed_assignments[var] = value
            else:
                remaining_clauses.append(clause)

        return remaining_clauses, fixed_assignments

    def apply_fixed_assignments(
        clauses: List[List[Tuple[str, bool]]], fixed_assignments: Dict[str, bool]
    ) -> List[List[Tuple[str, bool]]]:
        """
        Simplify clauses based on fixed assignments.
        """
        simplified_clauses = []
        for clause in clauses:
            new_clause = []
            satisfied = False
            for var, value in clause:
                if var in fixed_assignments:
                    if fixed_assignments[var] == value:
                        satisfied = True
                        break
                else:
                    new_clause.append((var, value))
            if not satisfied and new_clause:
                simplified_clauses.append(new_clause)
        return simplified_clauses

    

    def solve(self) -> List[Dict[str, bool]]:
        """
        Solve the SAT problem using grouped clauses and fixed assignment enforcement.
        """
        try:
            clauses, fixed_assignments = self.enforce_fixed_assignments(self.clauses)
            clauses = self.apply_fixed_assignments(clauses, fixed_assignments)

            grouped_clauses = self.group_clauses_by_size()
            solutions = self.solve_group(grouped_clauses[0])

            for group in grouped_clauses[1:]:
                group_solutions = self.solve_group(group)
                solutions = self.merge_assignments(solutions, group_solutions)

                if not solutions:
                    return []

            # Combine fixed assignments with found solutions
            for solution in solutions:
                solution.update(fixed_assignments)
            return solutions
        except ValueError as e:
            # Handle contradictory assignments
            print(f"Error: {e}")
            return []



# Example Usage
def main():
    clauses = [
        [('x1', True)],
        [('x1', True), ('x2', True), ('x3', True), ('x4', True), ('x5', True)],
        [('x1', False), ('x2', False), ('x3', False), ('x4', True), ('x5', False)],
        [('x1', True), ('x2', False), ('x3', True), ('x5', True)],
        [('x2', False), ('x3', True)],
    ]

    solver = SATSolver(clauses)
    solutions = solver.solve()

    print("Satisfiable Solutions:")
    for solution in solutions:
        print(OrderedDict(sorted(solution.items())))


if __name__ == "__main__":
    main()


TypeError: SATSolver.enforce_fixed_assignments() takes 1 positional argument but 2 were given

In [53]:
from typing import List, Tuple, Dict
from collections import OrderedDict

class SATSolver:
    def __init__(self, clauses: List[List[Tuple[str, bool]]]):
        """
        Initialize the SAT solver with the given clauses.
        Each clause is a list of tuples, where each tuple represents a literal as (variable, value).
        Example: [('x1', True), ('x2', False)] represents (x1 OR ¬x2).
        """
        self.clauses = clauses

    def enforce_fixed_assignments(
        self, clauses: List[List[Tuple[str, bool]]]
    ) -> Tuple[List[List[Tuple[str, bool]]], Dict[str, bool]]:
        """
        Extract and enforce single literal clauses as fixed assignments.
        """
        fixed_assignments = {}
        remaining_clauses = []

        for clause in clauses:
            if len(clause) == 1:  # Single-literal clause
                var, value = clause[0]
                if var in fixed_assignments and fixed_assignments[var] != value:
                    # Contradictory fixed assignments
                    raise ValueError(f"Contradictory assignment for variable {var}.")
                fixed_assignments[var] = value
            else:
                remaining_clauses.append(clause)

        return remaining_clauses, fixed_assignments

    def apply_fixed_assignments(
        self, clauses: List[List[Tuple[str, bool]]], fixed_assignments: Dict[str, bool]
    ) -> List[List[Tuple[str, bool]]]:
        """
        Simplify clauses based on fixed assignments.
        Remove satisfied clauses and reduce others by eliminating fixed literals.
        """
        simplified_clauses = []
        for clause in clauses:
            new_clause = []
            satisfied = False
            for var, value in clause:
                if var in fixed_assignments:
                    if fixed_assignments[var] == value:
                        satisfied = True
                        break
                else:
                    new_clause.append((var, value))
            if not satisfied and new_clause:
                simplified_clauses.append(new_clause)
        return simplified_clauses

    def group_clauses_by_size(self) -> List[List[List[Tuple[str, bool]]]]:
        """
        Group clauses by their size (number of literals) for structured solving.
        """
        clause_groups = {}
        for clause in self.clauses:
            size = len(clause)
            if size not in clause_groups:
                clause_groups[size] = []
            clause_groups[size].append(clause)

        return [clause_groups[key] for key in sorted(clause_groups)]

    def solve_group(self, group: List[List[Tuple[str, bool]]]) -> List[Dict[str, bool]]:
        """
        Solve a group of clauses of the same size by brute force.
        """
        if not group:
            return [{}]  # No clauses, trivially satisfied.

        solutions = []

        # Generate all possible truth assignments for the variables in this group
        variables = set(var for clause in group for var, _ in clause)
        n = len(variables)
        variable_list = list(variables)

        for i in range(1 << n):  # 2^n possible combinations
            assignment = {}
            for j in range(n):
                assignment[variable_list[j]] = bool(i & (1 << j))

            # Check if the assignment satisfies all clauses in the group
            if all(any(assignment[var] == value for var, value in clause) for clause in group):
                solutions.append(assignment)

        return solutions

    def merge_assignments(
        self, assignments1: List[Dict[str, bool]], assignments2: List[Dict[str, bool]]
    ) -> List[Dict[str, bool]]:
        """
        Merge two lists of assignments, ensuring compatibility.
        """
        merged = []

        for a1 in assignments1:
            for a2 in assignments2:
                merged_assignment = a1.copy()
                compatible = True

                for key, value in a2.items():
                    if key in merged_assignment and merged_assignment[key] != value:
                        compatible = False
                        break
                    merged_assignment[key] = value

                if compatible:
                    merged.append(merged_assignment)

        return merged

    def solve(self) -> List[Dict[str, bool]]:
        """
        Solve the SAT problem using grouped clauses and fixed assignment enforcement.
        """
        try:
            # Enforce fixed assignments and simplify clauses
            clauses, fixed_assignments = self.enforce_fixed_assignments(self.clauses)
            clauses = self.apply_fixed_assignments(clauses, fixed_assignments)

            # Update the solver's clauses
            self.clauses = clauses

            grouped_clauses = self.group_clauses_by_size()
            try:
                print(grouped_clauses)
                solutions = self.solve_group(grouped_clauses[0])
            except ValueError as e:
            # Handle contradictory assignments
                print(f"Error: {e}")
                return []

            for group in grouped_clauses[1:]:
                group_solutions = self.solve_group(group)
                solutions = self.merge_assignments(solutions, group_solutions)

                if not solutions:
                    return []  # No satisfying assignments.

            # Combine fixed assignments with found solutions
            for solution in solutions:
                solution.update(fixed_assignments)

            return solutions
        except ValueError as e:
            # Handle contradictory assignments
            print(f"Error: {e}")
            return []

# Example usage
if __name__ == "__main__":
    clauses = [
        [('x4', True),('x3', True)],
        [('x1', True), ('x2', True),('x3', True), ('x4', True),('x5', True)], 
        [('x1', False), ('x2', False),('x3', False), ('x4', True),('x5', False)],   # x1 OR (NOT x2)
        [('x1', True), ('x2', False),('x3', True), ('x5', True)],    # x2 OR x3
        [('x2', False), ('x3', True)],
        [('x2', False), ('x5', False)],
    ]

    solver = SATSolver(clauses)
    solutions = solver.solve()

    print("Satisfiable Solutions:")
    for solution in solutions:
        print(OrderedDict(sorted(solution.items())))


[[[('x4', True), ('x3', True)], [('x2', False), ('x3', True)], [('x2', False), ('x5', False)]], [[('x1', True), ('x2', False), ('x3', True), ('x5', True)]], [[('x1', True), ('x2', True), ('x3', True), ('x4', True), ('x5', True)], [('x1', False), ('x2', False), ('x3', False), ('x4', True), ('x5', False)]]]
Satisfiable Solutions:
OrderedDict([('x1', False), ('x2', False), ('x3', False), ('x4', True), ('x5', False)])
OrderedDict([('x1', True), ('x2', False), ('x3', False), ('x4', True), ('x5', False)])
OrderedDict([('x1', False), ('x2', False), ('x3', False), ('x4', True), ('x5', True)])
OrderedDict([('x1', True), ('x2', False), ('x3', False), ('x4', True), ('x5', True)])
OrderedDict([('x1', False), ('x2', False), ('x3', True), ('x4', False), ('x5', False)])
OrderedDict([('x1', True), ('x2', False), ('x3', True), ('x4', False), ('x5', False)])
OrderedDict([('x1', False), ('x2', False), ('x3', True), ('x4', True), ('x5', False)])
OrderedDict([('x1', True), ('x2', False), ('x3', True), ('x4

In [57]:
import itertools
from collections import defaultdict
import multiprocessing

# Utility function to evaluate a Boolean formula with a given assignment
def evaluate_formula(formula, assignment):
    for clause in formula:
        # Evaluate the clause
        clause_satisfied = False
        for literal in clause:
            if literal > 0 and assignment[abs(literal) - 1]:
                clause_satisfied = True
                break
            elif literal < 0 and not assignment[abs(literal) - 1]:
                clause_satisfied = True
                break
        if not clause_satisfied:
            return False
    return True

# Group clauses by the number of literals
def group_clauses(formula):
    grouped = defaultdict(list)
    for clause in formula:
        grouped[len(clause)].append(clause)
    return grouped

# Function to simplify the formula by removing satisfied clauses
def simplify_formula(formula, assignment):
    return [clause for clause in formula if not evaluate_formula([clause], assignment)]

# Parallel solver function for each group of clauses
def solve_group(group, formula, known_unsatisfiable_combinations):
    solutions = []
    
    # For each assignment of literals in the group, test it
    for assignment_combination in itertools.product([False, True], repeat=len(group)):
        # Simplify the formula for the current assignment
        current_assignment = [None] * len(formula)
        for i, assignment_value in enumerate(assignment_combination):
            current_assignment[i] = assignment_value
        
        simplified_formula = simplify_formula(formula, current_assignment)
        
        # Check if this solution is satisfiable
        if evaluate_formula(simplified_formula, current_assignment):
            solutions.append(current_assignment)
    
    return solutions

# Main SAT solver function
def solve_sat(formula):
    # Step 1: Group clauses based on their size
    grouped_clauses = group_clauses(formula)
    
    # Step 2: Set up a list to store satisfiable solutions and known unsatisfiable combinations
    satisfiable_solutions = []
    known_unsatisfiable_combinations = set()

    # Step 3: Solve each group and progressively test solutions
    for group_size in sorted(grouped_clauses.keys()):
        group = grouped_clauses[group_size]
        
        # Solve the group in parallel
        with multiprocessing.Pool() as pool:
            solutions = pool.apply(solve_group, (group, formula, known_unsatisfiable_combinations))
        
        # Prune solutions that lead to unsatisfiable combinations
        new_solutions = []
        for solution in solutions:
            if tuple(solution) not in known_unsatisfiable_combinations:
                new_solutions.append(solution)
                # Add this solution as a known unsatisfiable combination for future iterations
                known_unsatisfiable_combinations.add(tuple(solution))
        
        satisfiable_solutions.extend(new_solutions)
        
        # If no satisfiable solutions exist for this group, return unsatisfiable
        if not satisfiable_solutions:
            return "UNSATISFIABLE"
    
    # Step 4: After solving all groups, if we found satisfiable solutions, return the solutions
    if satisfiable_solutions:
        return satisfiable_solutions
    else:
        return "UNSATISFIABLE"

# Example formula (for testing)
# (A or B) and (not A or B) and (A or not B)
"""
    clauses = [
        [('x4', True),('x3', True)],
        [('x1', True), ('x2', True),('x3', True), ('x4', True),('x5', True)], 
        [('x1', False), ('x2', False),('x3', False), ('x4', True),('x5', False)],   # x1 OR (NOT x2)
        [('x1', True), ('x2', False),('x3', True), ('x5', True)],    # x2 OR x3
        [('x2', False), ('x3', True)],
        [('x2', False), ('x5', False)],
    ]
"""
formula = [
    [1,2,3,4,5],
    [-1,-2,-3,4,5],
    [1,-2,3,5],
    [-2,3],   # A or B
    [-2, 3],  # not A or B
    [-2,-5],  # A or not B
]

# Solve the SAT problem
result = solve_sat(formula)
for res in result:

    print(res)


[False, False, True, None, None, None]
[False, True, True, None, None, None]
[True, False, False, None, None, None]
[True, False, True, None, None, None]
[True, None, None, None, None, None]
[True, False, None, None, None, None]


In [60]:
import itertools
from collections import defaultdict
import multiprocessing

# Utility function to evaluate a Boolean formula with a given assignment
def evaluate_formula(formula, assignment):
    for clause in formula:
        clause_satisfied = False
        for literal in clause:
            if literal > 0 and assignment[abs(literal) - 1]:
                clause_satisfied = True
                break
            elif literal < 0 and not assignment[abs(literal) - 1]:
                clause_satisfied = True
                break
        if not clause_satisfied:
            return False
    return True

# Group clauses by the number of literals
def group_clauses(formula):
    grouped = defaultdict(list)
    for clause in formula:
        grouped[len(clause)].append(clause)
    return grouped

# Function to simplify the formula by removing satisfied clauses
def simplify_formula(formula, assignment):
    return [clause for clause in formula if not evaluate_formula([clause], assignment)]

# Parallel solver function for each group of clauses
def solve_group(group, formula, known_unsatisfiable_combinations):
    solutions = []
    
    # For each assignment of literals in the group, test it
    for assignment_combination in itertools.product([False, True], repeat=len(group)):
        current_assignment = [None] * len(formula)
        for i, assignment_value in enumerate(assignment_combination):
            current_assignment[i] = assignment_value
        
        simplified_formula = simplify_formula(formula, current_assignment)
        
        # Check if this solution is satisfiable
        if evaluate_formula(simplified_formula, current_assignment):
            solutions.append(current_assignment)
    
    return solutions

# Function to print the result in human-readable format
def format_solution(solution):
    # Convert the solution into a dictionary
    formatted_solution = {}
    for i, value in enumerate(solution):
        var_name = f'x{i + 1}'  # Variable name x1, x2, etc.
        if value is None:
            formatted_solution[var_name] = None
        else:
            formatted_solution[var_name] = value
    return formatted_solution

# Main SAT solver function
def solve_sat(formula):
    grouped_clauses = group_clauses(formula)
    
    satisfiable_solutions = []
    known_unsatisfiable_combinations = set()

    for group_size in sorted(grouped_clauses.keys()):
        group = grouped_clauses[group_size]
        
        with multiprocessing.Pool() as pool:
            solutions = pool.apply(solve_group, (group, formula, known_unsatisfiable_combinations))
        
        new_solutions = []
        for solution in solutions:
            if tuple(solution) not in known_unsatisfiable_combinations:
                new_solutions.append(solution)
                known_unsatisfiable_combinations.add(tuple(solution))
        
        satisfiable_solutions.extend(new_solutions)
        
        if not satisfiable_solutions:
            return "UNSATISFIABLE"
    
    if satisfiable_solutions:
        # Format and print the results in human-readable format
        formatted_solutions = [format_solution(solution) for solution in satisfiable_solutions]
        return formatted_solutions
    else:
        return "UNSATISFIABLE"

# Example formula
formula = [
    [-5],
    [1, 2, 3, 4, 5],
    [-1, -2, -3, 4, 5],
    [1, -2, 3, 5],
    [-2, 3],   # A or B
    [-2, -5],  # A or not B
]

# Solve the SAT problem
result = solve_sat(formula)
if result != "UNSATISFIABLE":
    for res in result:
        print(res)
else:
    print(result)


{'x1': True, 'x2': None, 'x3': None, 'x4': None, 'x5': None, 'x6': None}
{'x1': True, 'x2': False, 'x3': None, 'x4': None, 'x5': None, 'x6': None}


In [64]:
from itertools import combinations, product
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor
import threading

class SATSolver:
    def __init__(self, clauses):
        self.clauses = [tuple(clause) for clause in clauses]
        self.solutions = []
        self.impossible_constraints = set()
        self.lock = threading.Lock()

    def order_clauses(self):
        return sorted(self.clauses, key=len)

    def group_clauses(self, ordered_clauses):
        grouped = defaultdict(list)
        for clause in ordered_clauses:
            grouped[len(clause)].append(clause)
        return grouped

    def is_satisfiable(self, clause):
        # Check if the clause is satisfiable by any of the current solutions
        for solution in self.solutions:
            if any(lit in solution for lit in clause):
                return True
        return False

    def find_solutions(self, clause):
        # Find all possible solutions for the clause
        solutions = []
        for combination in product([True, False], repeat=len(clause)):
            if any(combination[i] == (lit > 0) for i, lit in enumerate(clause)):
                solutions.append(combination)
        return solutions

    def reduce_clauses(self, solutions, clauses):
        reduced = []
        for clause in clauses:
            if not any(all(lit in solution for lit in clause) for solution in solutions):
                reduced.append(clause)
        return reduced

    def solve_group_parallel(self, group):
        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(self.solve_clause, clause) for clause in group]
            solutions = [future.result() for future in futures]
        return solutions

    def solve_clause(self, clause):
        if not self.is_satisfiable(clause):
            with self.lock:
                self.impossible_constraints.add(clause)
            return []
        return self.find_solutions(clause)

    def explore_tree(self, solutions, clauses):
        if not clauses:
            return solutions
        next_group = self.group_clauses(clauses)[min(clauses, key=len)]
        new_solutions = self.solve_group_parallel(next_group)
        if not new_solutions:
            return []
        with self.lock:
            self.solutions.extend(new_solutions)
        reduced_clauses = self.reduce_clauses(new_solutions, clauses)
        return self.explore_tree(new_solutions, reduced_clauses)

    def solve(self):
        ordered_clauses = self.order_clauses()
        grouped_clauses = self.group_clauses(ordered_clauses)

        for length, group in sorted(grouped_clauses.items()):
            solutions = self.solve_group_parallel(group)
            if not solutions:
                return False
            with self.lock:
                self.solutions.extend(solutions)
            grouped_clauses = self.group_clauses(self.reduce_clauses(solutions, ordered_clauses))

        return self.explore_tree(self.solutions, ordered_clauses)

# Example usage
clauses = [
    [1, -2],
    [2, 3],
    [-1, -3],
    [1, 2, 3]
]

solver = SATSolver(clauses)
result = solver.solve()
print("SAT solvable:", result)


SAT solvable: []


In [67]:
from collections import defaultdict
from itertools import product
import concurrent.futures
from typing import List, Set, Dict, Tuple
import copy

class SATSolver:
    def __init__(self):
        self.clauses = []
        self.variables = set()
        self.impossible_combinations = set()
        self.simplified_nodes = {}
        self.solutions_cache = {}
        
    def add_clause(self, clause: List[int]):
        """Add a clause to the formula. Positive integers represent positive literals,
        negative integers represent negated literals."""
        self.clauses.append(set(clause))
        for literal in clause:
            self.variables.add(abs(literal))
            
    def group_clauses(self) -> Dict[int, List[Set[int]]]:
        """Group clauses by their size."""
        groups = defaultdict(list)
        for clause in self.clauses:
            groups[len(clause)].append(clause)
        return dict(sorted(groups.items()))
    
    def evaluate_assignment(self, assignment: Dict[int, bool], clause: Set[int]) -> bool:
        """Evaluate if a clause is satisfied by an assignment."""
        for literal in clause:
            var = abs(literal)
            if var in assignment:
                if (literal > 0 and assignment[var]) or (literal < 0 and not assignment[var]):
                    return True
        return False
    
    def solve_group(self, clauses: List[Set[int]], partial_assignment: Dict[int, bool] = None) -> List[Dict[int, bool]]:
        """Solve a group of clauses, considering partial assignment if provided."""
        if not clauses:
            return [{}]
            
        variables = set()
        for clause in clauses:
            for literal in clause:
                variables.add(abs(literal))
                
        if partial_assignment:
            variables = variables - set(partial_assignment.keys())
            
        assignments = []
        for values in product([False, True], repeat=len(variables)):
            assignment = dict(zip(variables, values))
            if partial_assignment:
                assignment.update(partial_assignment)
                
            # Check if this assignment contains known impossible combinations
            skip = False
            for impossible in self.impossible_combinations:
                if all(assignment.get(abs(lit)) == (lit > 0) for lit in impossible):
                    skip = True
                    break
            if skip:
                continue
                
            if all(self.evaluate_assignment(assignment, clause) for clause in clauses):
                assignments.append(assignment)
                
        return assignments
    
    def solve_parallel(self, group_solutions: Dict[int, List[Dict[int, bool]]]) -> List[Dict[int, bool]]:
        """Combine solutions from different groups in parallel."""
        def combine_solutions(sol1: Dict[int, bool], sol2: Dict[int, bool]) -> Dict[int, bool]:
            combined = sol1.copy()
            for var, val in sol2.items():
                if var in combined and combined[var] != val:
                    return None
                combined[var] = val
            return combined
            
        with concurrent.futures.ThreadPoolExecutor() as executor:
            current_solutions = group_solutions[min(group_solutions.keys())]
            for size in sorted(group_solutions.keys())[1:]:
                next_solutions = group_solutions[size]
                future_combinations = []
                
                for sol1 in current_solutions:
                    for sol2 in next_solutions:
                        future_combinations.append(
                            executor.submit(combine_solutions, sol1, sol2)
                        )
                
                current_solutions = [
                    f.result() for f in concurrent.futures.as_completed(future_combinations)
                    if f.result() is not None
                ]
                
                if not current_solutions:
                    return []
                    
        return current_solutions
    
    def simplify_node(self, node: Dict[int, bool]) -> str:
        """Create a unique key for a node to help with caching."""
        return ','.join(f"{k}:{int(v)}" for k, v in sorted(node.items()))
    
    def solve(self) -> List[Dict[int, bool]]:
        """Main solving method implementing the described algorithm."""
        grouped_clauses = self.group_clauses()
        group_solutions = {}
        
        # Solve each group in parallel
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future_to_size = {
                executor.submit(self.solve_group, clauses): size
                for size, clauses in grouped_clauses.items()
            }
            
            for future in concurrent.futures.as_completed(future_to_size):
                size = future_to_size[future]
                solutions = future.result()
                if not solutions:
                    # Found impossible constraint
                    return []
                group_solutions[size] = solutions
        
        # Combine solutions using parallel processing
        final_solutions = self.solve_parallel(group_solutions)
        
        # Update impossible combinations and simplified nodes
        if not final_solutions:
            for size, solutions in group_solutions.items():
                for sol in solutions:
                    node_key = self.simplify_node(sol)
                    if node_key not in self.simplified_nodes:
                        self.simplified_nodes[node_key] = set()
                    self.impossible_combinations.add(
                        frozenset((var if val else -var) for var, val in sol.items())
                    )
        
        return final_solutions

# Example usage
def example_usage():
    solver = SATSolver()
    formula=[
        [-5],
    [1, 2, 3, 4, 5],
    [-1, -2, -3, 4, 5],
    [1, -2, 3, 5],
    [-2, 3],   # A or B
    [-2, -5]]
    
    # Example: (a ∨ b) ∧ (¬b ∨ c) ∧ (¬a ∨ ¬c)
    """solver.add_clause([1, 2])        # a ∨ b
    solver.add_clause([-2, 3])       # ¬b ∨ c
    solver.add_clause([-1, -3])      # ¬a ∨ ¬c """
    
    for clause in formula:
        solver.add_clause(clause)
    solutions = solver.solve()
    
    if solutions:
        print("Satisfiable!")
        print("Solutions:")
        for solution in solutions:
            print({var: bool(val) for var, val in solution.items()})
    else:
        print("Unsatisfiable!")

if __name__ == "__main__":
    example_usage()

Satisfiable!
Solutions:
{5: False, 2: False, 3: True, 1: True, 4: False}
{5: False, 2: True, 3: True, 1: False, 4: True}
{5: False, 2: False, 3: False, 1: False, 4: True}
{5: False, 2: False, 3: True, 1: True, 4: True}
{5: False, 2: True, 3: True, 1: False, 4: False}
{5: False, 2: False, 3: True, 1: False, 4: False}
{5: False, 2: False, 3: True, 1: False, 4: True}
{5: False, 2: False, 3: False, 1: True, 4: False}
{5: False, 2: False, 3: False, 1: True, 4: True}
{5: False, 2: True, 3: True, 1: True, 4: True}


In [75]:
from collections import defaultdict
from itertools import product
import concurrent.futures
from typing import List, Set, Dict, Tuple, FrozenSet
import copy
from dataclasses import dataclass
import heapq

@dataclass
class Node:
    assignment: Dict[int, bool]
    children: Set['Node']
    impossible_patterns: Set[FrozenSet[int]]
    simplified: bool = False

class EnhancedSATSolver:
    def __init__(self):
        self.clauses = []
        self.variables = set()
        self.impossible_patterns = set()
        self.simplified_nodes = {}
        self.solutions_cache = {}
        self.pattern_cache = {}
        self.solution_tree = None
        
    def add_clause(self, clause: List[int]):
        self.clauses.append(frozenset(clause))
        for literal in clause:
            self.variables.add(abs(literal))

    def create_solution_tree(self, group_solutions: Dict[int, List[Dict[int, bool]]]) -> Node:
        """Create a tree structure of solutions, starting from groups with least solutions."""
        sorted_groups = sorted(group_solutions.items(), key=lambda x: len(x[1]))
        root = Node(assignment={}, children=set(), impossible_patterns=set())
        
        for size, solutions in sorted_groups:
            new_level = set()
            for solution in solutions:
                node = Node(solution, set(), set())
                compatible_parents = self._find_compatible_parents(root, solution)
                for parent in compatible_parents:
                    parent.children.add(node)
                new_level.add(node)
            
            # Simplify nodes at this level
            self._simplify_level_nodes(new_level)
        
        return root

    def _simplify_level_nodes(self, nodes: Set[Node]):
        """Simplify nodes that have identical patterns of satisfaction."""
        patterns = defaultdict(list)
        for node in nodes:
            pattern = self._get_satisfaction_pattern(node.assignment)
            patterns[pattern].append(node)
        
        # Merge nodes with same patterns
        for pattern, similar_nodes in patterns.items():
            if len(similar_nodes) > 1:
                merged_node = self._merge_nodes(similar_nodes)
                for node in similar_nodes:
                    node.simplified = True
                    node.children = merged_node.children

    def _get_satisfaction_pattern(self, assignment: Dict[int, bool]) -> FrozenSet[Tuple[int, bool]]:
        """Get a pattern of how clauses are satisfied by this assignment."""
        pattern_key = frozenset(assignment.items())
        if pattern_key in self.pattern_cache:
            return self.pattern_cache[pattern_key]
        
        pattern = []
        for clause in self.clauses:
            satisfaction = tuple(sorted(
                (literal, self._evaluate_literal(literal, assignment))
                for literal in clause
            ))
            pattern.append(satisfaction)
        
        result = frozenset(pattern)
        self.pattern_cache[pattern_key] = result
        return result

    def _merge_nodes(self, nodes: List[Node]) -> Node:
        """Merge similar nodes into a single simplified node."""
        merged_assignment = {}
        merged_impossible = set()
        
        for node in nodes:
            merged_impossible.update(node.impossible_patterns)
            for var, val in node.assignment.items():
                if var not in merged_assignment:
                    merged_assignment[var] = val
                elif merged_assignment[var] != val:
                    merged_assignment[var] = None  # Mark as don't care
        
        return Node(merged_assignment, set(), merged_impossible, simplified=True)

    def solve_group_parallel(self, clauses: List[FrozenSet[int]], batch_size: int = 1000) -> List[Dict[int, bool]]:
        """Solve a group of clauses in parallel with batched processing."""
        variables = set()
        for clause in clauses:
            for literal in clause:
                variables.add(abs(literal))
        
        all_assignments = list(product([False, True], repeat=len(variables)))
        batches = [all_assignments[i:i + batch_size] for i in range(0, len(all_assignments), batch_size)]
        
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future_to_batch = {
                executor.submit(self._process_batch, batch, variables, clauses): batch
                for batch in batches
            }
            
            solutions = []
            for future in concurrent.futures.as_completed(future_to_batch):
                solutions.extend(future.result())
        
        return solutions

    def _process_batch(self, batch, variables, clauses):
        """Process a batch of possible assignments."""
        solutions = []
        vars_list = sorted(variables)
        
        for values in batch:
            assignment = dict(zip(vars_list, values))
            if self._is_valid_assignment(assignment, clauses):
                solutions.append(assignment)
        
        return solutions

    def _is_valid_assignment(self, assignment: Dict[int, bool], clauses: List[FrozenSet[int]]) -> bool:
        """Check if an assignment is valid and doesn't contain known impossible patterns."""
        assignment_pattern = frozenset((var, val) for var, val in assignment.items())
        
        if assignment_pattern in self.impossible_patterns:
            return False
            
        return all(self._evaluate_clause(clause, assignment) for clause in clauses)

    def optimize_solution_tree(self):
        """Optimize the solution tree by identifying and caching common patterns."""
        if not self.solution_tree:
            return
            
        visited = set()
        queue = [(0, self.solution_tree)]
        
        while queue:
            level, node = heapq.heappop(queue)
            
            if id(node) in visited:
                continue
                
            visited.add(id(node))
            pattern = self._get_satisfaction_pattern(node.assignment)
            
            if pattern in self.simplified_nodes:
                node.simplified = True
                node.children = self.simplified_nodes[pattern].children
            else:
                self.simplified_nodes[pattern] = node
                
            for child in node.children:
                heapq.heappush(queue, (level + 1, child))

    def solve(self) -> List[Dict[int, bool]]:
        """Enhanced main solving method."""
        grouped_clauses = self.group_clauses()
        group_solutions = {}
        
        # Solve groups in parallel with batched processing
        for size, clauses in grouped_clauses.items():
            solutions = self.solve_group_parallel(clauses)
            if not solutions:
                return []
            group_solutions[size] = solutions
        
        # Create and optimize solution tree
        self.solution_tree = self.create_solution_tree(group_solutions)
        self.optimize_solution_tree()
        
        # Extract solutions from tree
        return self._extract_solutions(self.solution_tree)

    def _extract_solutions(self, root: Node) -> List[Dict[int, bool]]:
        """Extract all valid solutions from the solution tree."""
        solutions = []
        visited = set()
        
        def dfs(node: Node, current_assignment: Dict[int, bool]):
            if id(node) in visited:
                return
            
            visited.add(id(node))
            merged_assignment = current_assignment.copy()
            merged_assignment.update(node.assignment)
            
            if not node.children:
                if self._is_valid_assignment(merged_assignment, self.clauses):
                    solutions.append(merged_assignment)
                return
            
            for child in node.children:
                dfs(child, merged_assignment)
        
        dfs(root, {})
        return solutions
# Example usage
def example_usage():
    solver = SATSolver()
    formula=[
    [1, 2, 3, 4, 5],
    [-1, -2, -3, -4, -5],
    [1, -2, 3, 5],
    [-2, 3],   # A or B
    [-2, -5]
    ]
    
    # Example: (a ∨ b) ∧ (¬b ∨ c) ∧ (¬a ∨ ¬c)
    """solver.add_clause([1, 2])        # a ∨ b
    solver.add_clause([-2, 3])       # ¬b ∨ c
    solver.add_clause([-1, -3])      # ¬a ∨ ¬c """
    
    for clause in formula:
        solver.add_clause(clause)
    solutions = solver.solve()
    
    if solutions:
        print("Satisfiable!")
        print("Solutions:")
        for solution in solutions:
            print({var: bool(val) for var, val in sorted(solution.items())})
    else:
        print("Unsatisfiable!")

if __name__ == "__main__":
    example_usage()

Satisfiable!
Solutions:
{1: True, 2: False, 3: True, 4: True, 5: True}
{1: False, 2: False, 3: True, 4: True, 5: True}
{1: False, 2: False, 3: True, 4: False, 5: True}
{1: True, 2: False, 3: True, 4: False, 5: True}
{1: True, 2: False, 3: True, 4: False, 5: False}
{1: True, 2: False, 3: True, 4: True, 5: False}
{1: False, 2: False, 3: False, 4: True, 5: False}
{1: True, 2: False, 3: False, 4: False, 5: False}
{1: True, 2: False, 3: False, 4: True, 5: False}
{1: False, 2: False, 3: True, 4: False, 5: False}
{1: False, 2: False, 3: True, 4: True, 5: False}
{1: False, 2: False, 3: False, 4: False, 5: True}
{1: False, 2: False, 3: False, 4: True, 5: True}
{1: False, 2: True, 3: True, 4: False, 5: False}
{1: False, 2: True, 3: True, 4: True, 5: False}
{1: True, 2: False, 3: False, 4: False, 5: True}
{1: True, 2: False, 3: False, 4: True, 5: True}
{1: True, 2: True, 3: True, 4: False, 5: False}
{1: True, 2: True, 3: True, 4: True, 5: False}


In [77]:
from collections import defaultdict
from typing import List, Dict, Set, FrozenSet, Optional
from dataclasses import dataclass
import heapq
from concurrent.futures import ThreadPoolExecutor

@dataclass
class Node:
    solutions: Dict[int, bool]
    children: Set['Node']
    impossible_patterns: Set[FrozenSet[int]]
    group_id: int
    
class OptimizedSATSolver:
    def __init__(self):
        self.clauses_by_size = defaultdict(list)
        self.group_solutions = defaultdict(list)
        self.impossible_patterns = set()
        self.simplified_nodes = {}
        self.explored_combinations = {}
        
    def add_clause(self, clause: List[int]) -> None:
        """Add clause and organize by size"""
        frozen_clause = frozenset(clause)
        self.clauses_by_size[len(clause)].append(frozen_clause)

    def solve_group_parallel(self, group_clauses: List[FrozenSet[int]]) -> List[Dict[int, bool]]:
        """Solve a group of clauses in parallel"""
        variables = set()
        for clause in group_clauses:
            for literal in clause:
                variables.add(abs(literal))
        
        def test_assignment(assignment: Dict[int, bool]) -> Optional[Dict[int, bool]]:
            if self._violates_impossible_patterns(assignment):
                return None
            if all(self._is_clause_satisfied(clause, assignment) for clause in group_clauses):
                return assignment
            return None
        
        assignments = self._generate_assignments(variables)
        solutions = []
        
        with ThreadPoolExecutor() as executor:
            results = executor.map(test_assignment, assignments)
            solutions = [sol for sol in results if sol is not None]
            
        return solutions

    def _violates_impossible_patterns(self, assignment: Dict[int, bool]) -> bool:
        """Check if assignment contains known impossible patterns"""
        assignment_set = frozenset(assignment.items())
        return any(pattern.issubset(assignment_set) for pattern in self.impossible_patterns)

    def _is_clause_satisfied(self, clause: FrozenSet[int], assignment: Dict[int, bool]) -> bool:
        """Check if clause is satisfied by assignment"""
        return any(
            (literal > 0 and assignment.get(abs(literal), False)) or
            (literal < 0 and not assignment.get(abs(literal), True))
            for literal in clause
        )

    def _generate_assignments(self, variables: Set[int]) -> List[Dict[int, bool]]:
        """Generate all possible assignments for variables"""
        if not variables:
            return [{}]
        
        var = min(variables)
        remaining = variables - {var}
        sub_assignments = self._generate_assignments(remaining)
        
        result = []
        for sub in sub_assignments:
            for val in [False, True]:
                new_assignment = sub.copy()
                new_assignment[var] = val
                result.append(new_assignment)
        return result

    def _simplify_nodes(self, nodes: List[Node]) -> Node:
        """Simplify nodes with same satisfaction patterns"""
        pattern_groups = defaultdict(list)
        
        for node in nodes:
            pattern = self._get_satisfaction_pattern(node.solutions)
            pattern_groups[pattern].append(node)
        
        simplified_nodes = []
        for pattern, group in pattern_groups.items():
            if len(group) > 1:
                merged = self._merge_nodes(group)
                simplified_nodes.append(merged)
            else:
                simplified_nodes.extend(group)
        
        return simplified_nodes

    def _get_satisfaction_pattern(self, assignment: Dict[int, bool]) -> FrozenSet[tuple]:
        """Get pattern of how clauses are satisfied"""
        pattern = []
        for size, clauses in self.clauses_by_size.items():
            for clause in clauses:
                satisfaction = tuple(sorted(
                    (lit, self._evaluate_literal(lit, assignment))
                    for lit in clause
                ))
                pattern.append(satisfaction)
        return frozenset(pattern)

    def _evaluate_literal(self, literal: int, assignment: Dict[int, bool]) -> bool:
        """Evaluate a literal under an assignment"""
        var = abs(literal)
        if var not in assignment:
            return False
        return assignment[var] if literal > 0 else not assignment[var]

    def _merge_nodes(self, nodes: List[Node]) -> Node:
        """Merge similar nodes into a simplified node"""
        merged_solutions = {}
        merged_impossible = set()
        group_id = nodes[0].group_id
        
        for node in nodes:
            merged_impossible.update(node.impossible_patterns)
            for var, val in node.solutions.items():
                if var not in merged_solutions:
                    merged_solutions[var] = val
        
        return Node(merged_solutions, set(), merged_impossible, group_id)

    def solve(self) -> List[Dict[int, bool]]:
        """Main solving method following the described approach"""
        # Sort clauses by size
        sorted_groups = sorted(self.clauses_by_size.items(), key=lambda x: x[0])
        
        # Solve groups in order
        current_solutions = []
        for group_id, (size, clauses) in enumerate(sorted_groups):
            # Solve current group
            group_solutions = self.solve_group_parallel(clauses)
            if not group_solutions:
                return []  # No solution possible
                
            self.group_solutions[group_id] = group_solutions
            
            # Create nodes for current solutions
            current_nodes = [
                Node(sol, set(), set(), group_id)
                for sol in group_solutions
            ]
            
            # Simplify nodes
            simplified = self._simplify_nodes(current_nodes)
            
            # Update impossible patterns
            self._update_impossible_patterns(simplified)
            
            # Test against previous solutions
            if current_solutions:
                new_solutions = []
                for prev_sol in current_solutions:
                    for node in simplified:
                        combined = self._combine_solutions(prev_sol, node.solutions)
                        if combined and not self._violates_impossible_patterns(combined):
                            new_solutions.append(combined)
                current_solutions = new_solutions
            else:
                current_solutions = [node.solutions for node in simplified]
            
            if not current_solutions:
                return []
        
        return current_solutions

    def _update_impossible_patterns(self, nodes: List[Node]) -> None:
        """Update impossible patterns from nodes"""
        for node in nodes:
            self.impossible_patterns.update(node.impossible_patterns)

    def _combine_solutions(self, sol1: Dict[int, bool], sol2: Dict[int, bool]) -> Optional[Dict[int, bool]]:
        """Combine two partial solutions if compatible"""
        combined = sol1.copy()
        for var, val in sol2.items():
            if var in combined and combined[var] != val:
                return None
            combined[var] = val
        return combined
def example_usage():
    solver = SATSolver()
    formula=[
    [1, 2, 3, 4, 5],
    [-1, -2, -3, -4, -5],
    [1, -2, 3, 5],
    [-2, 3,4],   # A or B
    [-2, -5]
    ]
    
    # Example: (a ∨ b) ∧ (¬b ∨ c) ∧ (¬a ∨ ¬c)
    """solver.add_clause([1, 2])        # a ∨ b
    solver.add_clause([-2, 3])       # ¬b ∨ c
    solver.add_clause([-1, -3])      # ¬a ∨ ¬c """
    
    for clause in formula:
        solver.add_clause(clause)
    solutions = solver.solve()
    
    if solutions:
        print("Satisfiable!")
        print("Solutions:")
        for solution in solutions:
            print({var: bool(val) for var, val in sorted(solution.items())})
    else:
        print("Unsatisfiable!")

if __name__ == "__main__":
    example_usage()

Satisfiable!
Solutions:
{1: True, 2: True, 3: True, 4: True, 5: False}
{1: False, 2: False, 3: True, 4: True, 5: True}
{1: False, 2: True, 3: True, 4: False, 5: False}
{1: False, 2: False, 3: False, 4: True, 5: False}
{1: False, 2: False, 3: True, 4: False, 5: False}
{1: False, 2: False, 3: False, 4: False, 5: True}
{1: False, 2: False, 3: True, 4: False, 5: True}
{1: True, 2: False, 3: True, 4: True, 5: False}
{1: True, 2: False, 3: False, 4: True, 5: False}
{1: True, 2: False, 3: False, 4: False, 5: True}
{1: False, 2: False, 3: True, 4: True, 5: False}
{1: True, 2: False, 3: True, 4: True, 5: True}
{1: True, 2: False, 3: True, 4: False, 5: True}
{1: False, 2: False, 3: False, 4: True, 5: True}
{1: True, 2: False, 3: False, 4: True, 5: True}
{1: False, 2: True, 3: True, 4: True, 5: False}
{1: True, 2: False, 3: False, 4: False, 5: False}
{1: True, 2: False, 3: True, 4: False, 5: False}
{1: True, 2: True, 3: False, 4: True, 5: False}
{1: True, 2: True, 3: True, 4: False, 5: False}
